# Transform: Hourly Summary Table of NYC Taxi Trips - Data Transformation

This notebook aggregates raw NYC Yellow Taxi data (January 2023) into a structured hourly summary table, which can be used for visualization and downstream analytics. DuckDB is used for efficient SQL-based transformation.

In [1]:
# Imports and configuration
import pandas as pd
import duckdb
from pathlib import Path

# Paths
PROJECT_ROOT = Path("..")
RAW_DATA_PATH = PROJECT_ROOT / "data" / "raw" / "yellow_tripdata_2023-01.parquet"
OUTPUT_PATH = PROJECT_ROOT / "data" / "processed" / "trip_summary_hourly.csv"

In [2]:
# Connect to DuckDB and register the Parquet file as a virtual table
con = duckdb.connect(database=":memory:")
con.execute(f"CREATE VIEW yellow_taxi AS SELECT * FROM '{RAW_DATA_PATH}'")

### Load and Execute SQL Query

The transformation logic is stored in a separate `.sql` file for better readability and maintainability.

### SQL Transformation

The query is stored externally in a `.sql` file for better maintainability. It generates a summary table with hourly-level metrics: trip count, average distance, total amount, and duration (in minutes).

In [3]:
# Read SQL query from .sql file, query is loaded from external file for clarity
query_path = PROJECT_ROOT / "sql" / "trip_summary_hourly.sql"
with open(query_path) as f:
    query = f.read()

# Execute query and load results
df_summary = con.execute(query).fetchdf()
df_summary.head()

,pickup_hour,trip_count,avg_distance,avg_total_amount,avg_duration_min
0,2023-01-01 00:00:00,5217,3.388792,27.435576,18.045454
1,2023-01-01 01:00:00,5561,3.345114,27.218675,17.673581
2,2023-01-01 02:00:00,4944,3.441210,26.793956,17.534884
3,2023-01-01 03:00:00,3804,3.606159,27.200815,16.449330
4,2023-01-01 04:00:00,2381,3.977761,28.495120,15.952233


### Export

The resulting summary is saved as a CSV file under the processed data folder.

In [4]:
# Output the SQL querying results to csv files
df_summary.to_csv(OUTPUT_PATH, index=False)

print("Summary table generated and saved:", OUTPUT_PATH.resolve())

Summary table generated and saved: D:\DataTribes Four-Week Learning\data-engineering-learnings\tuo-yang-nyc-taxi\data\processed\trip_summary_hourly.csv


### Output Validation

Basic checks to ensure the summary table has the expected structure and no missing values.

In [5]:
# Sanity checks
expected_rows = 24 * 31
actual_rows = len(df_summary)
null_count = df_summary.isnull().sum().sum()

print(f"Row check: {actual_rows} / {expected_rows}")
print(f"Missing values: {null_count}")

assert actual_rows == expected_rows # Excepting 744 rows (hourly data for 31 days)
assert null_count == 0              # No missing values
print("Output shape and null check passed.")

Row check: 744 / 744
Missing values: 0
Output shape and null check passed.


### Summary

Hourly-level statistics have been successfully computed and saved. The output summary table can be used as a backend for visualizing key metrics such as trip volume, distance, and fare patterns over time.

Next step: build an interactive interface or dashboard to explore these trends more effectively — either within a web app (e.g., Streamlit) or using BI tools like Looker Studio.